In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter

In [2]:
flatten = lambda l:[item for sublist in l for item in sublist]
#1차원으로 펴주기

In [3]:
USE_CUDA = torch.cuda.is_available()
gpus = [0]
torch.cuda.set_device(gpus[0])

In [4]:
FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [5]:
def getBatch(batch_size, train_data):  #batch size: 한번에 학습시킬 데이터 크기
    random.shuffle(train_data) 
    sindex=0
    eindex = batch_size
    
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex + batch_size  #batch_size가 두배?
        sindex = temp #기존의 batch_size
        yield batch
        
    if eindex <= len(train_data):
        batch = train_data[sindex:]
        yield batch

## Data load and Preprocessing
### Gutenberg corpus

In [6]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [8]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:300]
#sampling sentences for test

In [9]:
corpus = [[word.lower() for word in sent] for sent in corpus] 
#전부 소문자로 변형

### Extract Stopwords

In [10]:
word_count = Counter(flatten(corpus))
print(len(word_count))

1484


In [11]:
border = int(len(word_count)*0.01) #1%
stopwords = word_count.most_common()[:border] + list(reversed(word_count.most_common()))[:border]

word_count.most_common() -> 빈도 높은 순서대로 정렬

list(reversed(word_count.most_common())) -> 빈도 낮은 순서대로 정렬 (리스트 형식)

In [12]:
stopwords
#빈도 가장 높은 상위 1% 단어들 + 빈도 가장 낮은 하위 1% 단어들 리스트

[(',', 284),
 ('the', 266),
 ('.', 192),
 ('of', 144),
 ('and', 112),
 ('"', 106),
 ('a', 105),
 ('in', 99),
 ('--', 94),
 ('to', 91),
 ('."', 86),
 ('whale', 76),
 ("'", 71),
 ('s', 66),
 ('feelings', 1),
 ('cherish', 1),
 ('degree', 1),
 ('knew', 1),
 ('surprising', 1),
 ('quietly', 1),
 ('throws', 1),
 ('cato', 1),
 ('flourish', 1),
 ('philosophical', 1),
 ('pistol', 1),
 ('substitute', 1),
 ('hats', 1),
 ('knocking', 1)]

In [13]:
stopwords = [s[0] for s in stopwords] # 단어만 확인
stopwords

[',',
 'the',
 '.',
 'of',
 'and',
 '"',
 'a',
 'in',
 '--',
 'to',
 '."',
 'whale',
 "'",
 's',
 'feelings',
 'cherish',
 'degree',
 'knew',
 'surprising',
 'quietly',
 'throws',
 'cato',
 'flourish',
 'philosophical',
 'pistol',
 'substitute',
 'hats',
 'knocking']

### Build vocab

In [14]:
vocab = list(set(flatten(corpus)) - set(stopwords))
vocab.append('<UNK>')
print(vocab[0:10])
#중복 처리를 위해 set 설정 이후 다시 리스트화
#전체 코퍼스 중에서 빈도 상,하위 1% 단어들(stopwords) 제거

['nantucket', 'wallen', 'punish', 'account', 'thrown', 'teeth', '1729', 'opening', 'things', 'stone']


In [15]:
print(len(set(flatten(corpus))))
print(len(vocab))

1484
1457


In [16]:
word2index = {'<UNK>':0} #vocab 생성 시작

for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
#만약 단어가 vocab 안에 없으면 vocab에 새로운 index 단어로 추가

In [17]:
index2word = {v:k for k,v in word2index.items()}

### Prepare train data

In [18]:
WINDOW_SIZE = 3
windows = flatten(list(nltk.ngrams(['<DUMMY>']*WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE,
                                  WINDOW_SIZE*2+1)) for c in corpus)
print(windows[0])
print(windows[1])

('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by')
('<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman')


nltk.ngrams(word_list, n)

In [22]:
train_data=[]
for window in windows:
    for i in range(WINDOW_SIZE*2+1):
        if i == WINDOW_SIZE or window[i] == '<DUMMY>': #중심 단어이거나 dummy 값일 경우
            continue
        train_data.append((window[WINDOW_SIZE], window[i])) #append(중심단어, 주변단어)
        
print(train_data[:WINDOW_SIZE*2+1])

[('[', 'moby'), ('[', 'dick'), ('[', 'by'), ('moby', '['), ('moby', 'dick'), ('moby', 'by'), ('moby', 'herman')]


In [33]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w:word2index[w] if word2index.get(w) is not None else word2index['<UNK>'], seq))
    return Variable(LongTensor(idxs))

def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]))

In [34]:
X_p=[]
y_p=[]

In [35]:
for tr in train_data:  
    X_p.append(prepare_word(tr[0], word2index).view(1,-1)) #중심단어 
    y_p.append(prepare_word(tr[1], word2index).view(1,-1)) #주변단어

a = Variable(torch.LongTensor(word2index[ '[' ]))

print(len(a))

In [27]:
train_data[0]

('[', 'moby')

In [28]:
train_data = list(zip(X_p, y_p))

print(train_data[0])
print('='*40)
print(len(train_data))

(Variable containing:
 1322
[torch.cuda.LongTensor of size 1x1 (GPU 0)]
, Variable containing:
 412
[torch.cuda.LongTensor of size 1x1 (GPU 0)]
)
26122


### Modeling

In [29]:
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(Skipgram, self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim) # v : 중심단어 벡터
        self.embedding_u = nn.Embedding(vocab_size, projection_dim) # u : 주변단어 벡터
        
        self.embedding_v.weight.data.uniform_(-1,1) #init
        self.embedding_u.weight.data.uniform_(0,0) #init
        #self.out = nn.Linear(projection_dim, vocab_size)
        
    def forward(self, center_words, target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B X 1 X D  #중심단어
        target_embeds = self.embedding_u(target_words) # B X 1 X D  #주변단어 중 하나 
        outer_embeds = self.embedding_u(outer_words) # B X V X D  #주변단어 
        
        scores = target_embeds.bmm(center_embeds.transpose(1,2)).squeeze(2) # BX1XD * BXDX1  => BX1
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1,2)).squeeze(2) #BXVXD * BXDX1  => BXV
        #bmm: 일괄행렬 계산
        
        nll = -torch.mean(torch.log(torch.exp(scores)/torch.sum(torch.exp(norm_scores),1).unsqueeze(1))) # log-softmax
        #negative log likelihood: 손실함수
        
        return nll # negative log likeligood
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds

#### torch.uniform(a,b) -> (a,b) 균일분포에 따라 임의의 실수를 구한다?

#### torch.squeeze() -> 차원의 크기가 1인 차원을 제거
#### torch.unsqueeze() -> 차원의 크기가 1인 차원을 원하는 차원에 생성

### Train

In [30]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 100

In [31]:
losses = []
model = Skipgram(len(word2index), EMBEDDING_SIZE)

if USE_CUDA:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01) #lr = learning rate

In [32]:
for epoch in range(EPOCH):
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs) # B X 1
        targets = torch.cat(targets) #B X 1
        vocabs = prepare_sequence(list(vocab), word2index).expand(inputs.size(0), len(vocab)) #B X V
        model.zero_grad()
        
        loss = model(inputs, targets, vocabs)
        
        loss.backward()
        optimizer.step()
        losses.append(loss.data.tolist()[0])
        
    if epoch % 10 == 0:
        print('Epoch: %d, mean_loss: %.02f' % (epoch,np.mean(losses)))
        losses=[]

Epoch: 0, mean_loss: 6.18
Epoch: 10, mean_loss: 4.33
Epoch: 20, mean_loss: 3.74
Epoch: 30, mean_loss: 3.64
Epoch: 40, mean_loss: 3.60
Epoch: 50, mean_loss: 3.59
Epoch: 60, mean_loss: 3.58
Epoch: 70, mean_loss: 3.57
Epoch: 80, mean_loss: 3.56
Epoch: 90, mean_loss: 3.56


#### torch.cat() -> 두개의 텐서 붙이기 (차원의 크기가 같아야 한다.)
dim=0 : 아래로    /    dim=1 : 오른쪽으로

### Test

In [36]:
def word_similarity(target, vocab):
    if USE_CUDA:
        target_V = model.prediction(prepare_word(target, word2index))
    else:
        target_V = model.prediction(prepare_word(target, word2index))
    similarities = []
    for i in range(len(vocab)):
        
        if vocab[i] == target: continue
            
        if USE_CUDA:
            vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        else:
            vector - model.prediction(prepare_word(list(vocab)[i], word2index))
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0]
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x:x[1], reverse=True)[:10] # sort by similarity

In [40]:
test = random.choice(list(vocab))
test

'erromangoan'

In [41]:
word_similarity(test,vocab)

[['fegee', 0.9464694857597351],
 ['pekee', 0.922804057598114],
 ['nuee', 0.9158511757850647],
 ['whoel', 0.7985355257987976],
 ['saxon', 0.7896506786346436],
 ['-', 0.7540516257286072],
 ['heads', 0.736070990562439],
 ['piggledy', 0.7126785516738892],
 ['grub', 0.7061753273010254],
 ['bone', 0.705224871635437]]